In [40]:
import numpy as np

In [41]:
def solve_sle_jacobi(A: np.ndarray, b: np.ndarray, x0: np.ndarray, epsilon: float, i_max: int) -> np.ndarray:
    """
    Solves the system of linear equations Ax = b using Jacobi method

    Example
    --------
    >>> A = np.array([[10, 1, 2], [4, 6, -1], [-2, 3, 8]], dtype=float)
    >>> b = np.array([[3, 9, 51]], dtype=float).T
    >>> x0 = np.array([[3/10, 9/6, 51/8]], dtype=float).T
    >>> solve_sle_jacobi(A, b, x0, 1e-8, 100)
    array([[-1.],
           [ 3.],
           [ 5.]])

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    b (ndarray) : nx1 constant matrix
    x0 (ndarray) : nx1 initial guess
    epsilon (float) : tolerance
    i_max (int) : maximum number of iterations

    Returns
    -------
    x (ndarray) : nx1 solution vector
    """
    m, n = np.shape(A)
    x = np.copy(x0)
    for k in range(i_max):
        x = np.concatenate((x, np.zeros((n, 1))), axis=1) # To store the values of x at each iteration
        for i in range(n):
            # if not abs(A[i,i]) > sum(abs(A[i,j]) for j in range(n) if i != j):
                # raise InvalidInputMatrix("The matrix is not diagonally dominant")
            sum_x = 0
            for j in range(n):
                if i != j:
                    sum_x += A[i, j] * x[j, k]
            x[i, k+1] = 1/A[i, i]*(b[i,0]-sum_x)
        verror = np.linalg.norm(x[:,k+1]-x[:,k])
        if verror < epsilon:
            return x[:, k+1][:, None]
    return x[:, i_max][:, None]

- Genera una matriz $\mathbf{A}\in \mathbb{R}^{100 \times 100}$ de forma aleatoria en Python y utilizando la función de numpy para factorización QR, obtén la matriz $\mathbf{Q}$ de la factorización $\mathbf{A=QR}$.

In [42]:
A = np.random.rand(4,4)
Q, _ = np.linalg.qr(A)
print(A)
print(Q)

[[0.64615898 0.09264093 0.87047399 0.3957063 ]
 [0.7895208  0.96121817 0.53150696 0.37770392]
 [0.54833551 0.91200666 0.61301375 0.86000806]
 [0.29374581 0.28313332 0.66871445 0.62876185]]
[[-0.54075707  0.79224033 -0.03126729 -0.28099716]
 [-0.66073361 -0.27190751  0.54002423  0.4448272 ]
 [-0.45889063 -0.54622471 -0.32518242 -0.62073695]
 [-0.24582979  0.00775443 -0.77566267  0.58125296]]


- Define una matriz diagonal $\mathbf{D}$, tal que $d_{ii}>d_{(i+1)(i+1)}$ y $d_{11}<1$.

In [43]:
def gen_d(n: int=100) -> np.ndarray:
    """
    Generates a diagonal matrix D with random values
    """
    D = np.zeros((n,n))
    D[0,0] = np.random.random() # D[0,0] < 1
    for i in range(n-1):
        D[i+1,i+1] = (np.random.random())*D[i,i] # D[i,i] > D[i+1,i+1]
    return D

In [44]:
D = gen_d(4)
D

array([[0.32189289, 0.        , 0.        , 0.        ],
       [0.        , 0.0869225 , 0.        , 0.        ],
       [0.        , 0.        , 0.06512773, 0.        ],
       [0.        , 0.        , 0.        , 0.05431774]])

- Con la matriz $\mathbf{Q}$ y la matriz $\mathbf{D}$, define una nueva matriz $$\mathbf{M=QDQ^T}$$ Como $\rho(M)<1$, se puede aproximar la solución a la ecuación $\mathbf{Mx=b}$ con Jacobi y Gauss-Seidel.

In [45]:
M = Q@D@Q.T
M

array([[0.15303636, 0.08839751, 0.05239882, 0.03603236],
       [0.08839751, 0.1766958 , 0.08407422, 0.0388649 ],
       [0.05239882, 0.08407422, 0.12153497, 0.0327734 ],
       [0.03603236, 0.0388649 , 0.0327734 , 0.07699374]])

In [46]:
# Condicion suficiente de convergencia
for i in range(4):
    print(f'Row {i}: {sum(abs(M[i,j]) for j in range(4) if j != i)}')
    if not abs(M[i,i]) > sum(abs(M[i,j]) for j in range(4) if j != i):
            print(f'|M[{i,i}]|={abs(M[i,i])} Puede no converger ')

Row 0: 0.1768286912863266
|M[(0, 0)]|=0.15303636367148327 Puede no converger 
Row 1: 0.21133662900286937
|M[(1, 1)]|=0.17669579509236868 Puede no converger 
Row 2: 0.16924643941351436
|M[(2, 2)]|=0.12153497118945056 Puede no converger 
Row 3: 0.10767066327908373
|M[(3, 3)]|=0.0769937364435238 Puede no converger 


In [47]:
# Condicion necesaria de convergencia
for i in range(4):
    for j in range(4):
        if not abs(M[i,i]) > abs(M[i,j]) and i != j:
            print(f'No converge: |M[{i,j}]| > |M[{i,i}]|')

- Genera un vector $\mathbf{x}$ y $\mathbf{b}$ como en el ejercicio 1(e), utilizando $\mathbf{M}$ en lugar de $\mathbf{A}$.

In [48]:
x_s = np.array([[i+1 for i in range(4)]], dtype=float).T
b = M@x_s

In [49]:
x_jacobi = solve_sle_jacobi(M, b, np.zeros((4,1)), 1e-8, 100)

In [50]:
x_jacobi.flatten()

array([-3.73491477e+10, -3.85324878e+10, -4.30731954e+10, -4.36531157e+10])